# Code Breaking Part : Affine Cipher

The Affine cipher is a monoalphabetic substitution cipher 

The Affine cipher encrypts messages using a mathematical key. 

- Let the length of the alphabet be $\ell$
- The two numbers in the key are $a$ and $b$, where:
$$1 \leq a \leq \ell  \qquad 1 \leq b \leq \ell $$
- $a$ should be relativley prime to $\ell$ (That is $a$ and $\ell$ should not share any factors)
eg) if $\ell=26$ then $a$ should not have any factors of $2$ or $13$

$$ a \in \{1,3,5,7,9,11,15,17,19,21,23,25 \} $$
$$ b \in \{0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25 \} $$

Let $p$ be the position of the plain letter in the plain alphaabet. ie for English, these are the values of $p$

$p$ | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10| 11| 12 |13 |14 |15 |16| 17|18 |19 |20 |21 |22 |23 |24| 25|
---| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---| ---| --- |--- |--- |--- |---| ---|--- |--- |--- |--- |--- |--- |---| ---|
Plain Alphabet| a |b | c |d | e | f | g | h | i | j | k | l | m |n |o |p |q |r |s |t |u |v |w |x |y |z | 

**Encryption Function**
$$ c = ap + b (\mod m) $$

**Decryption Function**
$$ p = a^{-1}(c - b) (\mod \ell) $$

... where $a a^{-1} = 1 (\mod \ell)$

In [1]:
from Resources.Functions import *

In [57]:
def findFactors(x):
    '''A function to find all of the factors of a given number'''
    factors = []
    for i in range(1, x + 1):
        if x % i == 0:
            factors.append(i)
    return factors


def findAffineKeys(plainAlphabet = englishAlphabet):

    b = list(np.arange(0,len(plainAlphabet)))

    factorsAlphabetLength = findFactors(len(plainAlphabet))

    a = []
    for number in np.arange(1,len(plainAlphabet)):
        factorsNumber = findFactors(number)
        inBoth = [i for i in factorsNumber if i in factorsAlphabetLength and i !=1]
        if len(inBoth) == 0:
            a.append(number)

    return a, b

## Step 1) Encrypt the message

In [7]:
def Incrypt_Affine( plainText, a, b, plainAlphabet=englishAlphabet ):

    key = {}
    for p in range(len(plainAlphabet)):
        c = (a*p + 7)%len(plainAlphabet)
        key[plainAlphabet[p].upper()] = plainAlphabet[c].upper()

    cipherText = ''
    for character in plainText:
        if character.isalpha():
            cipherText = cipherText + key[character.upper()]
        else:
            cipherText = cipherText + character
            
    return cipherText

In [16]:
Incrypt_Affine('defend the east wall of the castle', 3, 7)

'QTWTUQ MCT THJM VHOO XW MCT NHJMOT'

## Step 2) Decrypt the message

In [13]:
def Decrypt_Affine( cipherText, a, b, plainAlphabet=englishAlphabet ):

    key = {}
    for c in range(len(plainAlphabet)):
        a_invrt = pow(a,-1,len(plainAlphabet))
        p = a_invrt * (c-b) % len(plainAlphabet)
        key[plainAlphabet[c].upper()] = plainAlphabet[p].upper()

    plainText = ''
    for character in cipherText:
        if character.isalpha():
            plainText += key[character.upper()]
        else:
            plainText += character
            
    return plainText

In [18]:
Decrypt_Affine('QTWTUQ MCT THJM VHOO XW MCT NHJMOT', 3, 7)

'DEFEND THE EAST WALL OF THE CASTLE'

## Step 3) Break an Affine Cipher

There are a limited number of possible keys that can be used to encrypt a message using an Affine Cipher 

For english: $12*26 = 312$ possible keys

In order to break this encryption method all we need to do is check every possible key and see how close to english the resulting texct is 

In [41]:
def break_Affine( cipherText, quadgramsFreq=quadgramFreq_English, plainAlphabet=englishAlphabet ):
    a, b = findAffineKeys(plainAlphabet)
    allKeys = [(int(x),int(y)) for x in a for y in b]
    
    bestKey = []
    bestScore = -99e9
    for key in allKeys:
        decryptTest = Decrypt_Affine(cipherText, key[0],key[1], plainAlphabet)
        score = quadgramFitness(decryptTest, quadgramsFreq)
        if score > bestScore:
            bestKey = key
            bestScore = score
    
    return bestKey, Decrypt_Affine(cipherText,bestKey[0],bestKey[1], plainAlphabet)
    
    

In [42]:
break_Affine("QUVNLAUVILZKVZZZVNHIVQUFSFZHWZQLQHQLJSNLAUVIFZLQLZYHKSVIFWKVQJFKKJMQUVFQQFNTZQUFQPJITFDFLSZQZHWZQLQHQLJSNLAUVIZLSFEELQLJSQJJQUVIFQQFNTZ")

((17, 5),
 'THECIPHERISLESSSECURETHANASUBSTITUTIONCIPHERASITISVULNERABLETOALLOFTHEATTACKSTHATWORKAGAINSTSUBSTITUTIONCIPHERSINADDITIONTOOTHERATTACKS')